<a href="https://colab.research.google.com/github/aquapathos/BasicAI/blob/master/5CNN/CIFAR10PyTorch002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color=red>**【共有中に付き編集不可】**　改変したい場合はコピーを作成してください　PR,IAI</font>

In [ ]:
# @title
# 動作環境の確認

import datetime
import pytz

def 時刻表示(phase):
    japan_tz = pytz.timezone('Asia/Tokyo')
    current_japan_time = datetime.datetime.now(japan_tz)
    formatted_time = current_japan_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
    print(phase,formatted_time)
    return formatted_time
時刻表示('開始時刻')

!pip -q install deep-translator requests
!pip -q install imagehash
!pip -q install icrawler deep-translator nltk torch transformers pillow

import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
if device.type != 'cpu':
  print(torch.cuda.get_device_name())
  print(torch.cuda.get_device_capability())

# 画像の収集
画像を認識するためには、まず事前にたくさんの事例をAIに機械学習させておく必要がありますが、
大量の画像を自分で用意するのは大変なので、プログラムで収集することにします。

以下、重要な指示は<font color='blue'>青字</font>，特に重要な指示は<font color='red'>赤字</font>にしてありますので、見落とさないように演習を進めていってください。

# 1. Google Drive をマウント
Google Colaboratory は一定時間経過すると作業内容が削除されてしまいます。消したくないデータや後日再利用したいデータは Google Drive に保存するようにプログラムすることで、残すことができます。


1. <font color='blue'>この説明の下のセルを実行する
2. アクセス許可の確認ダイアログが出る。「Googleドライブに接続」をクリック。
3. アカウントを選択する。
4. アクセスリクエストのダイアログに変わる。一番下までスクロールし「許可」をクリック。
5. 左のファイル一覧エリアで「更新アイコン」（回転マーク）をクリック（エリアが表示されていないなら「ファイルアイコン」（フォルダ型）をクリック）</font>


<img width="390" alt="googlecolab" src="https://user-images.githubusercontent.com/5820803/94802343-739cff00-0422-11eb-8c0d-affa919f8e58.png">

　<font color=blue>次のセルを実行してください</font>




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 2. 画像の収集と選別

自分で収集した画像を使って独自のデータセットを作り、先と同様の実験をやってみましょう。

## 2-1 画像の収集

画像を収集するために必要なライブラリを準備します。  <font color='blue'>次のセルを実行してください</font>

In [ ]:
#@title [１] ライブラリ準備
# ==============================================================================
# 1. ライブラリ準備
# ==============================================================================
import os
import random
import shutil
import time
from pathlib import Path
import glob
import matplotlib.pyplot as plt

import nltk
from deep_translator import GoogleTranslator
from icrawler.builtin import BingImageCrawler
from PIL import Image
import imagehash
import numpy as np
import cv2
import torch

from transformers import CLIPModel, CLIPProcessor
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import wordnet

# from icrawler.builtin import GoogleImageCrawler
import warnings
import datetime
warnings.simplefilter('ignore')

ViewSIZE = 128 # 表示用画像サイズ高さと幅
SIZE = 32 # モデルに入力する際の画像サイズ

機械学習の際に画像のサイズをすべて同じに揃えます。ViewSIZEは表示用の画像サイズ、SIZEは学習・認識時のサイズです。  
CIFAR10に合わせるためにSIZE = 32としています。

次に、収集する画像のカテゴリを決めましょう。



--
CIFAR-10 のカテゴリは
- airplane (飛行機)
- automobile (自動車）
- bird (鳥類）
- cat （ネコ)
- deer (シカ）
- dog （イヌ）
- frog （カエル）
- horse（ウマ）
- ship（船舶）
- truck（トラック）

でした。比較したいので、<font color=red>**ここに含まれるの中から「ネコ」とネコ以外のカテゴリを１つ**</font>、さらに、ここに<font color=red>**含まれないカテゴリを２つ**</font>，計４つのカテゴリを考えましょう。
以下では、CIFAR10のカテゴリから「ネコ」と「イヌ」、追加カテゴリとして「カメ」「スクータ」を選んだものとして説明していきます。

## **<font color=red>確認 実験に使う４つのカテゴリ</font>**
・「ネコ」を４つのうちの一つとして必ず含める．  
・CIFAR-10の10カテゴリから，ネコ以外に一つ選ぶ．  
・上に含まれないカテゴリを２つ，計４カテゴリを考える．

下のセルのカテゴリ名を自分で選んだ４カテゴリに書き換えて実行してください。  

  <font color='blue'>次のセルを実行してください</font>

In [ ]:
#@title  [２]  設定 & 収集カテゴリ定義
# ==============================================================================
# 2. 設定 & AIクラス定義
# ==============================================================================
# 収集したいカテゴリ  （次の行は例です。'ネコ'以外を書き換えてください）
myclasses = ['ネコ','イヌ','灯台','スクータ']

# 保存先設定
target_num = 300          # 収集目標
TMPFOLDER = 'drive/MyDrive/tmp/'

# オプション設定
USE_CLIP_FILTER = True
CLIP_THRESHOLD = 0.22   # 0.20〜0.25推奨

# 掛け合わせ用ワード（背景やアングルを変えて枚数を稼ぐ魔法の言葉）
ACTIONS_AND_CONTEXTS = [
    "photo", "outside", "close up", "face", "running",
    "street", "scenery", "colorful", "side view", "beautiful", "landscape"
]


ネット上で公開されている画像を収集するためのライブラリ、icrawler を使って画像を収集します。 今回は Bingから画像を拾ってくることにします。（以前はGoogle検索を使っていましたが，仕様変更で使えなくなりました．）

次のセルは、上で選んだカテゴリの画像をそれぞれ３００枚を目標として収集し、一旦 Google Drive のマイドライブ内の 'tmp'という名のフォルダに保存するプログラムです。tmpフォルダ内にカテゴリごとのサブフォルダができます。（ "drive/MyDrive" が Google Driveのマイドライブを表しています）

  <font color='blue'>次のセルを実行してください</font> （定義だけなので何も起こりません）

In [ ]:
#@title 画像収集エージェントの定義
# ---------------------------------------------------------------
# 初期準備
# ---------------------------------------------------------------
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')
    nltk.download('omw-1.4')

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 軽量テキスト埋め込みモデル
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=DEVICE)

# CLIPモデル
clip_model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32').to(DEVICE)
clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')

translator = GoogleTranslator(source='auto', target='en')

# Haar Cascade（顔検出）
haar_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_path)

# グローバルキュー
GLOBAL_SEARCH_QUEUE = {}

# アクション定義
ACTIONS_AND_CONTEXTS = [
    'photo', 'close up', 'side view', 'on white background', 'in context', 'in use'
]

# ---------------------------------------------------------------
# StrategyPlanner
# ---------------------------------------------------------------
class StrategyPlanner:
    def __init__(self, similarity_threshold=0.7, max_candidates=40):
        self.to_en = translator
        self.similarity_threshold = similarity_threshold
        self.max_candidates = max_candidates

    def _clip_text_sim(self, base, cands):
        base_emb = embedder.encode(base, convert_to_tensor=True)
        cand_embs = embedder.encode(cands, convert_to_tensor=True)
        sims = util.cos_sim(base_emb, cand_embs)[0].cpu().numpy()
        return sims.tolist()

    def get_subcategories(self, category_jp):
        print(f"🔍 '{category_jp}' の関連語を生成中... (類似度閾値={self.similarity_threshold})")

        try:
            cat_en = self.to_en.translate(category_jp)
        except Exception:
            cat_en = category_jp

        # WordNet 展開
        cand = set()
        cand.add(cat_en.lower())
        for syn in wordnet.synsets(cat_en):
            for lemma in syn.lemmas():
                cand.add(lemma.name().replace('_', ' ').lower())
            for hypo in syn.hyponyms():
                name = hypo.name().split('.')[0].replace('_', ' ').lower()
                cand.add(name)

        cand = [c for c in sorted(cand) if 1 < len(c) <= 30 and c.isprintable()]
        cand = cand[:self.max_candidates]

        sims = self._clip_text_sim(cat_en, cand)

        def is_valid_candidate(w):
            base_tokens = set(cat_en.lower().split())
            w_tokens = set(w.lower().split())
            return bool(base_tokens & w_tokens)

        filtered = [w for w, s in zip(cand, sims) if s >= self.similarity_threshold and is_valid_candidate(w)]

        if category_jp not in filtered:
            filtered.insert(0, category_jp)

        if not filtered:
            filtered = [cat_en]

        print(f"   -> 候補: {filtered[:10]} ... 計 {len(filtered)}")
        return filtered

# ---------------------------------------------------------------
# ImageFilterAI / ダウンロード / 重複排除 / CLIPフィルタ関数
# ---------------------------------------------------------------
class ImageFilterAI:
    def __init__(self):
        if torch.cuda.is_available():
            self.device = "cuda"
            self.batch_size = 24
            self.dtype = torch.float16
        else:
            self.device = "cpu"
            self.batch_size = 4
            self.dtype = torch.float32

        self.model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(self.device, dtype=self.dtype)
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        self.translator = GoogleTranslator(source='auto', target='en')

    def _prepare_labels(self, category_text, negative_words):
        pos = self.translator.translate(category_text)
        negs = [self.translator.translate(n) for n in negative_words]
        pos_labels = [f"{pos}", f"a {pos}", f"a real photo of a {pos}"]
        neg_labels = [f"a {n}" for n in negs] + ["text", "watermark", "illustration", "cartoon"]
        return pos_labels + neg_labels

    def filter(self, folder_path, category_text, negative_words, threshold):
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg','.jpeg','.png'))]
        if not image_files: return
        labels = self._prepare_labels(category_text, negative_words)

        deleted = 0
        for i in range(0, len(image_files), self.batch_size):
            batch_files = image_files[i:i+self.batch_size]
            images = []
            for fname in batch_files:
                path = os.path.join(folder_path, fname)
                try:
                    with Image.open(path) as img:
                        images.append(img.convert("RGB"))
                except:
                    os.remove(path)
                    deleted += 1
            if not images: continue

            inputs = self.processor(text=labels, images=images, return_tensors="pt", padding=True).to(self.device)
            for k in ["pixel_values"]:
                inputs[k] = inputs[k].to(dtype=self.dtype)
            with torch.no_grad():
                outputs = self.model(**inputs)
                probs = outputs.logits_per_image.softmax(dim=1)

            pos_count = len(labels) // 2
            for fname, p in zip(batch_files, probs):
                path = os.path.join(folder_path, fname)
                pos_score = p[:pos_count].max().item()
                best_idx = p.argmax().item()
                if best_idx >= pos_count or pos_score < threshold:
                    try: os.remove(path); deleted += 1
                    except: pass

        print(f"🗑️ Removed {deleted} images.")

def download_images(queries, out_dir, max_per_query=50):
    os.makedirs(out_dir, exist_ok=True)
    for q in queries:
        try:
            crawler = GoogleImageCrawler(storage={'root_dir': out_dir})
            crawler.crawl(keyword=q, max_num=max_per_query)
        except Exception as e:
            print(f"  [WARN] ダウンロード失敗: {q} ({e})")

def remove_duplicates(folder_path, hash_threshold=5):
    files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    hashes = {}
    removed = 0
    for fname in files:
        path = os.path.join(folder_path, fname)
        try:
            with Image.open(path) as img:
                h = imagehash.phash(img.convert('RGB'))
        except:
            try: os.remove(path); removed += 1
            except: pass
            continue
        is_dup = False
        for old_h in list(hashes.keys()):
            if h - old_h <= hash_threshold:
                try: os.remove(path); removed += 1; is_dup=True; break
                except: pass
        if not is_dup: hashes[h] = fname
    print(f"   -> {removed} 枚削除されました")

# ---------------------------------------------------------------
# UI/インタラクション部分は直接セルで input() を使う前提
# - search_keyword_ui(), process_global_queue() は削除済み
# ---------------------------------------------------------------

GLOBAL_SEARCH_QUEUE = {}
print("✅ 準備完了。次のセルで UI や収集処理を実行できます")

# [3] 検索語の決定

検索したいものを見つけるためのキーワードと排除したいものを表すキーワードを決めます。

## 検索キーワードの修正
検索は英語を使った方が多くヒットするので、まず日本語を英語に変換し、類義語をAIに提案させます。

語のニュアンスが日本語と英語とでずれている場合があります。例えば、「スクータ」は英語では「scooter」ですが、英語の「scooter」には海上スクータが含まれます。

スクータ　→　scooter　→ sea scooter, motor scooter, water scooter

AIが検索語の候補を提案してきますので、不要なものは削除してください。

→　スクータ, scooter, motor scooter

## 検索キーワードの追加
もっとヒットする可能性のある語があれば追加してください。

→　スクータ, scooter, motor scooter, ホンダスクータ, Yamaha scooter

## 排除キーワード

検索画像には望ましくないものもたくさん含まれますので、これは含んでほしくないというものを聞いてきますので答えてください。

例）キックボード、人、ショップ，おもちゃ，人形

In [ ]:
#@title [３] 検索語の決定
# ==============================================================================
# 3. フェーズ1：人間によるキーワードの確定
# ==============================================================================
# 重複排除関数
def remove_duplicates(lst):
    seen = set()
    result = []
    for x in lst:
        if x not in seen:
            result.append(x)
            seen.add(x)
    return result

#@title [３] 検索語の決定（クリーン版）
時刻表示('検索語選択の開始')
planner = StrategyPlanner()
GLOBAL_SEARCH_QUEUE = {}  # リセット

print("\n" + "="*60)
print("📝 検索キーワードを計画しましょう")
print("="*60 + "\n")

for categ in myclasses:
    print(f"--- カテゴリ: 【 {categ} 】 ---")

    # AIに関連語を取得（日本語カテゴリ＋英語派生語）
    sub_cats = planner.get_subcategories(categ)
    print(f"🤖 AIの提案: {', '.join(sub_cats[:8])} ...など計 {len(sub_cats)} 個")

    # 学生に入力させる
    print("👇 絞り込みや追加があれば入力 (例，柴犬，チワワ，ホンダスクータ，ヤマハスクータ；空欄ならAI提案を全採用):")
    user_input = input(f">>> ")
    base_keywords = [w.strip() for w in user_input.replace('、', ',').replace('，', ',').split(',') if w.strip()]
    if not base_keywords:
        base_keywords = list(sub_cats)

    # 排除ワード
    print("🗑️ 排除したいもの (例: 人、ショップ、おもちゃ, doll, キックボード, イラスト):")
    neg_input = input(f">>> ")
    neg_words = [w.strip() for w in neg_input.replace('、', ',').replace('，', ',').split(',') if w.strip()]

    # 掛け合わせクエリの生成
    final_queries = []
    final_queries.extend(base_keywords)
    for base in base_keywords:
        for action in ACTIONS_AND_CONTEXTS:
            final_queries.append(f"{base} {action}")

    # 重複排除
    final_queries = remove_duplicates(final_queries)
    random.shuffle(final_queries)

    # グローバル変数に保存
    GLOBAL_SEARCH_QUEUE[categ] = {'queries': final_queries, 'neg': neg_words}
    print(f"✅ '{categ}' の検索パターンを {len(final_queries)} 通り作成しました。\n")

print("✨ 全カテゴリの設定完了！ 次のセルを実行して収集を始めましょう。")
時刻表示('検索語決定完了')

# [4] 自動収集と自動選別
Bing検索で指定されたキーワードで画像を集め、画像を認識してキーワードに合っていないものや排除対象が写っているものを取り除きます。

In [ ]:
#@title [４] 自動収集と自動選別
# ==============================================================================
# 4. フェーズ2：全自動実行（収集 ＋ CLIP選別）
# ==============================================================================
時刻表示('収集開始')

if not GLOBAL_SEARCH_QUEUE:
    print("⚠️ エラー: STEP 1 がまだ実行されていません。先に戻ってキーワードを決めてください。")
else:
    print("\n" + "="*60)
    print("🚀 画像収集スタート")
    print("="*60 + "\n")

    clip_filter = ImageFilterAI() if USE_CLIP_FILTER else None

    for categ, plan in GLOBAL_SEARCH_QUEUE.items():
        print(f"\n📂 カテゴリ処理中: {categ}")

        queries = plan['queries']
        neg_list = plan['neg']
        neg_str = " ".join([f"-{n}" for n in neg_list])

        dir_path = os.path.join(TMPFOLDER, categ)
        os.makedirs(dir_path, exist_ok=True)

        crawler = BingImageCrawler(storage={"root_dir": dir_path}, downloader_threads=4)

        query_index = 0
        while True:
            current_count = len(os.listdir(dir_path))

            # 目標の1.3倍でストップ
            if current_count >= target_num * 1.3:
                print(f"🛑 目標達成！収集を終了します (現在: {current_count}枚)")
                break

            if query_index >= len(queries):
                print("⚠️ 用意したクエリを使い切りました。これ以上は見つかりにくいです。")
                break

            q_word = queries[query_index]
            full_query = f"{q_word} {categ} {neg_str} -illustration"

            print(f"   [{current_count}枚] 🔍 検索: {full_query}")

            try:
                crawler.crawl(
                    keyword=full_query,
                    filters={'type': 'photo'},
                    max_num=50,
                    overwrite=False,
                    file_idx_offset='auto'
                )
            except Exception as e:
                print(f"   (エラーによりスキップ: {e})")

            query_index += 1
            time.sleep(1)

        # 選別フェーズ（CLIP）
        if clip_filter:
            clip_filter.filter(dir_path, categ, neg_list, CLIP_THRESHOLD)

        print(f"🎉 カテゴリ '{categ}' 完了: {len(os.listdir(dir_path))} 枚")

時刻表示('収集完了')

ERRORが出るのはリンク切れ(404)やアクセス制限(403)であり、プログラムのエラーではないので気にしなくても構いません。

## 2-2 画像の選別

![Google Drive](https://raw.githubusercontent.com/aquapathos/pictures/gh-pages/BasicAIFig001.png)　　　　　　 ![Google Drive](https://raw.githubusercontent.com/aquapathos/pictures/gh-pages/BasicAIFig002.png)   

![Google Drive](https://github.com/aquapathos/pictures/blob/gh-pages/BasicAIFig003.png?raw=true)

1. <font color='blue'>ブラウザで新しいタブをつくり、[ドライブ](https://drive.google.com/) を開きます。
2. **マイドライブ**の中の**tmp**フォルダを開いてみてください。 自分が決めたカテゴリのフォルダが並んでいるはずです。
3.「ネコ」を例として説明します。**ネコ** フォルダを開きます。
4. リストレイアウトからグリッドレイアウトに切り替えて不適だと思われる画像を削除してください。
5. 同様にして、ほかのカテゴリ（例ではイヌ、灯台、スクータ）についても不要画像を削除してください。</font>

不適なのは、次のような画像です。
- ネコが写っていない画像。写っていても小さすぎる画像
- 主要被写体がネコではない画像、つまりネコ以外のものが目立っている画像
- 文字が目立つ画像。できれば文字は入っていないことが望ましいですが、目立たなければOK
- デフォルメされたぬいぐるみやイラスト。
- GIF 動画
- 同一の写真

パッと見て、これは「ネコ」が主題の写真だと感じられるものを残して、そうでないもの、他のモノが多くの面積を占めているものは削除してください。(厳密なルールはありません。あなた自身がこれは何かと問われて「ネコ」と答える写真を選べばいいです。あなたの選んだものが教師データとなります。)

# 2-3　画像のリサイズ

何度も使うので関数を定義しておきます。  <font color='blue'>次のセルを実行してください。</font>　　（コードは非表示になっていますが、実行する必要があります。）

In [ ]:
#@title リサイズ関数の定義
def resize(foldername, size=ViewSIZE):
    imgnames = glob.glob(foldername+"/*") # 画像ファイル名のリスト
    images = []
    for imgname in imgnames:
        img = cv2.cvtColor(np.array(Image.open(imgname).convert('RGBA')),cv2.COLOR_RGBA2RGB)
        height = img.shape[0]
        width = img.shape[1]
        if height > width :
            m = (height - width)//2
            img = img[m:m+width]
        else:
            m = (width - height)//2
            img = img[:,m:m+height]
        img = cv2.resize(img, (size,size))
        images.append(img)
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        cv2.imwrite(imgname,img)
    return images

さっそくリサイズしてみましょう。　　<font color='blue'>次のセルを実行してください。</font>




In [ ]:
# @title
input('不適切画像の削除は終了したら Enterを押してください')
# リサイズを実行
images = []
for i,categ in enumerate(myclasses):
  images.append(resize(TMPFOLDER+categ))
  print(f"{categ} - {len(images[i])} 枚の画像がリサイズされました")
images = np.array(images, dtype=object) ## numpy 仕様変更あり 2024

表示用関数を定義し、表示して確認してみます。　　<font color='blue'>次のセルを実行してください。</font>　コードは非表示にしてあります。

In [ ]:
#@title 表示用関数の定義
import math
# start番からnpic枚表示する関数を定義
plt.rcParams['figure.figsize'] = (12.0, 7.0)
def showimg(images, start = 0, npic = 48):
    n = npic if len(images) >= start+npic else len(images) - start
    plt.figure(figsize=(8,7.5*(math.ceil(n/8))/6),dpi=150)
    i = 0
    while True:
        if i < n :
            plt.subplot((n-1)//8+1,8,i+1)
            plt.xticks([])
            plt.yticks([])
            plt.imshow(images[start+i])
            plt.title("{}".format(start+i))
            i += 1
        else:
            break

さっそく表示させてみましょう。　　<font color='blue'>次のセルを実行してください。</font>

In [ ]:
for i in range(len(images)):
  showimg(images = images[i],start=0,npic=8) # 各カテゴリ16枚ずつ表示

## 2-4 学習用画像データの保存と読み込み
python にはメモリ上に置かれた変数の値をそっくりそのままファイルとして保存する関数が用意されていますので、その関数を使って変数の中身を Google ドライブに保存したり、読み出したりできるようにしておきます。　　<font color='blue'>次のセルを実行してください。</font>


In [ ]:
#@title
import pickle
import os

# 画像データを pickle 形式で保存
def storeCategoryImages(cat, fname, folder = "."):
    os.makedirs(folder, exist_ok=True)
    f = open(folder+"/"+fname,'wb')
    pickle.dump(cat,f)
    f.close

# pickle 形式で保存された画像データの読み込み
def loadCategoryImages(fname, folder = "."):
    f = open(folder+"/"+fname,'rb')
    cat = pickle.load(f)
    f.close
    return cat

さっそくリサイズした画像をpickle 形式で保存しましょう。　　<font color='blue'>次のセルを実行してください。</font>

In [ ]:
GFOLDER = "drive/MyDrive/CIFAR10"  # データ保存用のフォルダ名
for i,categ in enumerate(myclasses):
  print(i,categ)
  storeCategoryImages(images[i],f"{categ}.pkl",folder=GFOLDER)

今度は逆に pickle 形式で保存されたデータを変数に読み込み、正しく復元できるか確認しておきましょう。一つ目のカテゴリだけ試してみます。　　<font color='blue'>次のセルを実行してください。</font>

In [ ]:
testimg = loadCategoryImages("ネコ.pkl", folder=GFOLDER)
showimg(images = testimg,start = 4,npic=8)

# CIFAR10PyTorch003 に進んでください。

------